In [ ]:
! pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
from tensorflow import keras
import keras_tuner as kt
from keras_tuner import HyperModel, RandomSearch, Hyperband

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
def build_model(hp):  # Defines a function that takes a HyperModel as input and returns a built Keras model.
  """
  This function builds a CNN architecture based on the hyperparameters provided by the HyperModel.
  """
  # Define input layer
  inputs = keras.Input(shape=(28, 28, 1))  # Creates an input layer for 28x28 grayscale images.

  # Convolutional layer block
  """
  This loop iterates a specified number of times (defined by the "conv_layers" hyperparameter)
  and builds a series of convolutional layers with various configurations.
  """
  x = inputs
  for _ in range(hp.Int("conv_layers", min_value=2, max_value=5)):
    x = keras.layers.Conv2D(  # Applies a 2D Convolutional layer with:
        hp.Int("filters", min_value=32, max_value=256),  # Number of filters (feature maps) specified by the "filters" hyperparameter.
        kernel_size=3,  # Kernel size of 3x3 for extracting local features.
        activation="relu",  # ReLU activation for non-linearity.
    )(x)  # Applies the layer to the previous layer output (x).

    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)  # Applies Max Pooling with a 2x2 window for feature reduction.
    x = keras.layers.Dropout(hp.Float("dropout", min_value=0.1, max_value=0.5))(x)  # Introduces dropout with a probability specified by the "dropout" hyperparameter for preventing overfitting.

  # Flatten and add dense layers
  x = keras.layers.Flatten()(x)  # Flattens the output of the convolutional layers into a 1D vector.
  """
  This loop iterates a specified number of times (defined by the "dense_layers" hyperparameter)
  and builds a series of fully-connected layers with various configurations.
  """
  for _ in range(hp.Int("dense_layers", min_value=2, max_value=5)):
    x = keras.layers.Dense(  # Applies a Dense layer with:
        hp.Int("neurons", min_value=32, max_value=256),  # Number of neurons specified by the "neurons" hyperparameter.
        activation="relu",  # ReLU activation for non-linearity.
    )(x)
    x = keras.layers.Dropout(hp.Float("dropout", min_value=0.1, max_value=0.5))(x)  # Introduces dropout with a probability specified by the "dropout" hyperparameter for preventing overfitting.

  # Output layer
  outputs = keras.layers.Dense(10, activation="softmax")(x)  # Creates a final Dense layer with 10 neurons (one for each digit) and softmax activation for classification.
  model = keras.Model(inputs=inputs, outputs=outputs)  # Creates a Keras model with the defined input and output layers.
  model.compile(  # Configures the model for training:
      optimizer=keras.optimizers.Adam(learning_rate=hp.Float("learning_rate", min_value=0.001, max_value=0.1)),  # Uses Adam optimizer with a learning rate specified by the "learning_rate" hyperparameter.
      loss="sparse_categorical_crossentropy",  # Defines the loss function for classification.
      metrics=["accuracy"],  # Specifies "accuracy" as the metric to track during training.
  )
  return model  # Returns the built and configured model.


In [ ]:
# Define hypermodel for tuning
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3)

# Train and search for best hyperparameters
tuner.search(x_train, y_train, epochs=10, validation_split=0.2)  # Trains the model and searches for the best hyperparameters using the provided training data, epochs, and validation split.

# Get best model and evaluate
best_model = tuner.get_best_models()[0]

Trial 22 Complete [00h 00m 00s]

Best val_accuracy So Far: 0.9460833072662354
Total elapsed time: 00h 08m 34s


In [ ]:
best_model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 4ms/step - loss: 0.2209 - accuracy: 0.9466


[0.22088539600372314, 0.9466000199317932]